In [1]:
#-*-coding:utf-8-*-
# =================================
# Python 3
# Jupyter Notebook 6.0.0
# Author: hqrrr
# Date of last update: Oct. 2020
# =================================

import numpy as np

# ====================
#      paramters
# ====================

richtung_WS = 1 # Direction of heat flow, 0: upwards, 1: horizontal, 2: downwards
r = 4 # U value accuracy
r0 = 6 # Accuracy of calculation process (recommend >=4)


# ====================
#         DIN
# ====================

# DIN EN ISO 6946, Tabelle 9
if richtung_WS == 0:
    R_si = 0.1
    R_se = 0.04
elif richtung_WS == 1:
    R_si = 0.13
    R_se = 0.04
elif richtung_WS == 2:
    R_si = 0.17
    R_se = 0.04
else:
    print('ERROR: direction of heat flow incorrect(0 ~ 2)')
    error += 1
    

# DIN EN ISO 6946, Tabelle 10
# [Dicke der Luftschicht [mm], Wärmedurchlasswiderstand R [m2k/W] in Richtung aufwärts, - in Richtung horizontal, - in Richtung abwärts]
# [thickness of the air layer [mm], thermal resistance R [m2k/W] upwards, - horizontally, - downwards]
air = np.array([
    [0,0,0,0],
    [5,0.11,0.11,0.11],
    [7,0.13,0.13,0.13],
    [10,0.15,0.15,0.15],
    [15,0.16,0.17,0.17],
    [25,0.16,0.18,0.19],
    [50,0.16,0.18,0.21],
    [100,0.16,0.18,0.22],
    [300,0.16,0.18,0.23]
])



# ====================
# input any float number into [Lambda] and ['Luftschicht'] into [Building material] if it's air layer(dormant)
# Default Lambda value of the air layer
Lambda_air = 0.1
# ====================
#        input
# ====================

p = 4 # number of parts of the building structure

# 1st part of the building structure
# Dicke [m] / Lambda [W/mK] / 'Baustoff'
# Thickness [m] / Lambda [W/mK] / 'Building material'
# input any float number into [Lambda] and ['Luftschicht'] into [Building material] if it's air layer(dormant)
m0 = np.array([
            [0.0125,0.21,'Gipskarton'],
            [0.0002,0.2,'PE-Flie'],
            [0.0125,0.21,'Gipskarton'],
            [0.01,0.04,'Mineralwolle'],
            [0.08,0.04,'Mineralwolle'],
            [0.01,0.04,'Mineralwolle'],
            [0.015,0.36,'Gipsfaser'],
            [0.06,0.035,'Faserdaemmstoffe']
            ])

m0_p = 0.84 # Percentage of components of the building structure

# 2nd part
# Dicke [m] / Lambda [W/mK] / 'Baustoff'
# Thickness [m] / Lambda [W/mK] / 'Building material'
# input any float number into [Lambda] and ['Luftschicht'] into [Building material] if it's air layer(dormant)
m1 = np.array([
            [0.0125,0.21,'Gipskarton'],
            [0.0002,0.2,'PE-Flie'],
            [0.0125,0.21,'Gipskarton'],
            [0.01,60,'Luftschicht'],
            [0.08,99999,'Luftschicht'],
            [0.01,60,'Stahl'],
            [0.015,0.36,'Gipsfaser'],
            [0.06,0.035,'Faserdaemmstoffe']
            ])

m1_p = 0.05 # Percentage of components of the building structure

# 3rd part
# Dicke [m] / Lambda [W/mK] / 'Baustoff'
# Thickness [m] / Lambda [W/mK] / 'Building material'
# input any float number into [Lambda] and ['Luftschicht'] into [Building material] if it's air layer(dormant)
m2 = np.array([
            [0.0125,0.21,'Gipskarton'],
            [0.0002,0.2,'PE-Flie'],
            [0.0125,0.21,'Gipskarton'],
            [0.01,60,'Luftschicht'],
            [0.08,60,'Stahl'],
            [0.01,60,'Stahl'],
            [0.015,0.36,'Gipsfaser'],
            [0.06,0.035,'Faserdaemmstoffe']
            ])

m2_p = 0.01 # Percentage of components of the building structure

# 4th part
# Dicke [m] / Lambda [W/mK] / 'Baustoff'
# Thickness [m] / Lambda [W/mK] / 'Building material'
# input any float number into [Lambda] and ['Luftschicht'] into [Building material] if it's air layer(dormant)
m3 = np.array([
            [0.0125,0.21,'Gipskarton'],
            [0.0002,0.2,'PE-Flie'],
            [0.0125,0.21,'Gipskarton'],
            [0.01,60,'Stahl'],
            [0.08,60,'Stahl'],
            [0.01,60,'Stahl'],
            [0.015,0.36,'Gipsfaser'],
            [0.06,0.035,'Faserdaemmstoffe']
            ])

m3_p = 0.1 # Percentage of components of the building structure


In [2]:
# reset value before calculating
error = 0
R_i = 0
list_R_a = []
list_R_b = []
list_R_c = []
list_R_d = []
list_R_tot_i = []
list_R_i = []
R_T = 0
R_tot_upper = 0
R_tot_lower = 0
U_value = 0



# ====================
#      functions
# ====================

# calculate Lambda of the air layer
def Lambda_airlayer(x,matrix):
    global Lambda_air
    global d
    global error
    R_air = 1
    
    if richtung_WS == 0:
        j = 1
    elif richtung_WS == 1:
        j = 2
    elif richtung_WS == 2:
        j = 3
    else:
        print('ERROR: direction of heat flow incorrect(0 ~ 2)')
        error += 1
    
    d_air = float(matrix[x][0]) * 1000 # [m] -> [mm]
        
    # calculate R value of the air layer
    if d_air < air[len(air) - 1][0]:
        for i in range (0,len(air)):
            if d_air >= air[i][0] and d_air < air[i+1][0]:
                # linear interpolation
                # (d - d[i]) / (R - R[i]) = (d[i+1] - d[i]) / (R[i+1] - R[i])
                # here d = d_airlayer // d[i] = air[i][0] // R = R_air // R[i] = air[i][j]
                R_air = air[i][j] + (d_air - air[i][0]) * (air[i+1][j] - air[i][j]) / (air[i+1][0] - air[i][0])
            else:
                pass
    elif d_air == air[len(air) - 1][0]:
        R_air = air[len(air) - 1][j]
        print('d_air == air[len(air) - 1][0]')
    else:
        print('ERROR: thickness of airlayer incorrect')
        error += 1
    
    Lambda_air = round(d_air / 1000 / R_air,r0)
    print('Air layer, x = ',x)
    print('R_air : ',R_air)
    print('d_air [mm] : ',d_air)
    print('Lambda_air : ',Lambda_air)
    print('-----------------------------------')
    return Lambda_air


# calculate U value
def calculate_U():
    global error
    global R_i
    global list_R_a
    global list_R_b
    global list_R_c
    global list_R_d
    global list_R_tot_i
    global list_R_i
    global R_T
    global R_tot_upper
    global R_tot_lower
    global U_value
    # homogen
    # homogeneous
    if p == 1:
        for i in range (0,len(m0)):
        # for each layer of the building structure
            list_R_a.append(round(float(m0[i][0]) / float(m0[i][1]),r0))
            # calculate R value (d / Lambda, here d = m0[i][0] & Lambda = m0[i][1]) and add it into the list_R
        R_T = round(R_si + R_se + sum(list_R_a),r0)
        U_value = round(1 / R_T,r0)
    
    # inhomogen
    # inhomogeneous
    elif p == 2:
        if m0_p + m1_p != 1:
            print('ERROR: sum of m_p != 1')
            error += 1
        else:
            for i in range (0,len(m0)):
                list_R_a.append(round(float(m0[i][0]) / float(m0[i][1]),r0))
                list_R_b.append(round(float(m1[i][0]) / float(m1[i][1]),r0))
                R_i = 1/(m0_p / list_R_a[i] + m1_p / list_R_b[i])
                list_R_i.append(round(R_i,r))
            list_R_tot_i.append(round(R_si + R_se + sum(list_R_a),r0))
            list_R_tot_i.append(round(R_si + R_se + sum(list_R_b),r0))
            R_tot_upper = 1 / (m0_p / list_R_tot_i[0] + m1_p / list_R_tot_i[1])
            R_tot_upper = round(R_tot_upper,r0)
            R_tot_lower = R_si + R_se + sum(list_R_i)
            R_tot_lower = round(R_tot_lower,r0)
            R_tot = round((R_tot_upper + R_tot_lower) / 2,r0)
            U_value = round(1 / R_tot,r)
    elif p == 3:
        if m0_p + m1_p + m2_p != 1:
            print('ERROR: sum of m_p != 1')
            error += 1
        else:
            for i in range (0,len(m0)):
                list_R_a.append(round(float(m0[i][0]) / float(m0[i][1]),r0))
                list_R_b.append(round(float(m1[i][0]) / float(m1[i][1]),r0))
                list_R_c.append(round(float(m2[i][0]) / float(m2[i][1]),r0))
                R_i = 1/(m0_p / list_R_a[i] + m1_p / list_R_b[i] + m2_p / list_R_c[i])
                list_R_i.append(round(R_i,r))
            list_R_tot_i.append(round(R_si + R_se + sum(list_R_a),r0))
            list_R_tot_i.append(round(R_si + R_se + sum(list_R_b),r0))
            list_R_tot_i.append(round(R_si + R_se + sum(list_R_c),r0))
            R_tot_upper = 1 / (m0_p / list_R_tot_i[0] + m1_p / list_R_tot_i[1] + m2_p / list_R_tot_i[2])
            R_tot_upper = round(R_tot_upper,r0)
            R_tot_lower = R_si + R_se + sum(list_R_i)
            R_tot_lower = round(R_tot_lower,r0)
            R_tot = round((R_tot_upper + R_tot_lower) / 2,r0)
            U_value = round(1 / R_tot,r)
    elif p == 4:
        if m0_p + m1_p + m2_p + m3_p != 1:
            print('ERROR: sum of m_p != 1')
            error += 1
        else:
            for i in range (0,len(m0)):
                list_R_a.append(round(float(m0[i][0]) / float(m0[i][1]),r0))
                list_R_b.append(round(float(m1[i][0]) / float(m1[i][1]),r0))
                list_R_c.append(round(float(m2[i][0]) / float(m2[i][1]),r0))
                list_R_d.append(round(float(m3[i][0]) / float(m3[i][1]),r0))
                R_i = 1/(m0_p / list_R_a[i] + m1_p / list_R_b[i] + m2_p / list_R_c[i] + m3_p / list_R_d[i])
                list_R_i.append(round(R_i,r))
            list_R_tot_i.append(round(R_si + R_se + sum(list_R_a),r0))
            list_R_tot_i.append(round(R_si + R_se + sum(list_R_b),r0))
            list_R_tot_i.append(round(R_si + R_se + sum(list_R_c),r0))
            list_R_tot_i.append(round(R_si + R_se + sum(list_R_d),r0))
            R_tot_upper = 1 / (m0_p / list_R_tot_i[0] + m1_p / list_R_tot_i[1] + m2_p / list_R_tot_i[2] + m3_p / list_R_tot_i[3])
            R_tot_upper = round(R_tot_upper,r0)
            R_tot_lower = R_si + R_se + sum(list_R_i)
            R_tot_lower = round(R_tot_lower,r0)
            R_tot = round((R_tot_upper + R_tot_lower) / 2,r0)
            U_value = round(1 / R_tot,r)
    else:
        print('ERROR: p value incorrect')
        error += 1
    return U_value
    


In [3]:
# check if there is any air layer, find its thickness and calculate its Lambda value
for x in range (0,len(m0)):
    if m0[x][2] == 'Luftschicht':
        m0[x][1] = Lambda_airlayer(x,m0)
        # x: x-th layer of the building structure
        # m0,m1,m2,m3 : Different parts of the building structure
for x in range (0,len(m1)):
    if m1[x][2] == 'Luftschicht':
        m1[x][1] = Lambda_airlayer(x,m1)

for x in range (0,len(m2)):
    if m2[x][2] == 'Luftschicht':
        m2[x][1] = Lambda_airlayer(x,m2)

for x in range (0,len(m3)):
    if m3[x][2] == 'Luftschicht':
        m3[x][1] = Lambda_airlayer(x,m3)

if len(m0) == len(m1) == len(m2) == len(m3):
    U_value = calculate_U()
else:
    print('ERROR: Matrix not match')
    error += 1

# ====================
#       output
# ====================
if error != 0:
    pass
else:
    if p == 1:
        print('matrix 0 : ',m0_p * 100,'%')
        print(m0)
        print('----------------------')
        print('length = ',len(m0))
    elif p == 2:
        print('matrix 0 : ',m0_p * 100,'%')
        print(m0)
        print('----------------------')
        print('matrix 1 : ',m1_p * 100,'%')
        print(m1)
        print('----------------------')
        print('length = ',len(m0),'/',len(m1))
    elif p == 3:
        print('matrix 0 : ',m0_p * 100,'%')
        print(m0)
        print('----------------------')
        print('matrix 1 : ',m1_p * 100,'%')
        print(m1)
        print('----------------------')
        print('matrix 2 : ',m2_p * 100,'%')
        print(m2)
        print('----------------------')
        print('length = ',len(m0),'/',len(m1),'/',len(m2))
    elif p == 4:
        print('matrix 0 : ',m0_p * 100,'%')
        print(m0)
        print('----------------------')
        print('matrix 1 : ',m1_p * 100,'%')
        print(m1)
        print('----------------------')
        print('matrix 2 : ',m2_p * 100,'%')
        print(m2)
        print('----------------------')
        print('matrix 3 : ',m3_p * 100,'%')
        print(m3)
        print('----------------------')
        print('length = ',len(m0),'/',len(m1),'/',len(m2),'/',len(m3))
    else:
        pass

    print('R_si = ',R_si,',','R_se = ',R_se)
    print('list_R_a : ',list_R_a)
    print('list_R_b : ',list_R_b)
    print('list_R_c : ',list_R_c)
    print('list_R_d : ',list_R_d)
    print('list_R_tot_i : ',list_R_tot_i)
    print('list_R_i : ',list_R_i)
    print('R_tot_upper : ',R_tot_upper,',','R_tot_lower : ',R_tot_lower)
    print('U value = ',U_value)

Air layer, x =  3
R_air :  0.15
d_air [mm] :  10.0
Lambda_air :  0.066667
-----------------------------------
Air layer, x =  4
R_air :  0.18
d_air [mm] :  80.0
Lambda_air :  0.444444
-----------------------------------
Air layer, x =  3
R_air :  0.15
d_air [mm] :  10.0
Lambda_air :  0.066667
-----------------------------------
matrix 0 :  84.0 %
[['0.0125' '0.21' 'Gipskarton']
 ['0.0002' '0.2' 'PE-Flie']
 ['0.0125' '0.21' 'Gipskarton']
 ['0.01' '0.04' 'Mineralwolle']
 ['0.08' '0.04' 'Mineralwolle']
 ['0.01' '0.04' 'Mineralwolle']
 ['0.015' '0.36' 'Gipsfaser']
 ['0.06' '0.035' 'Faserdaemmstoffe']]
----------------------
matrix 1 :  5.0 %
[['0.0125' '0.21' 'Gipskarton']
 ['0.0002' '0.2' 'PE-Flie']
 ['0.0125' '0.21' 'Gipskarton']
 ['0.01' '0.066667' 'Luftschicht']
 ['0.08' '0.444444' 'Luftschicht']
 ['0.01' '60' 'Stahl']
 ['0.015' '0.36' 'Gipsfaser']
 ['0.06' '0.035' 'Faserdaemmstoffe']]
----------------------
matrix 2 :  1.0 %
[['0.0125' '0.21' 'Gipskarton']
 ['0.0002' '0.2' 'PE-Flie']
